## Copy Disbursement + PLoan Script

### Disbursement

In [1]:
import shutil
import subprocess
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import *
from dateutil import parser
import warnings
import xlwings as xw
import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import numbers
warnings.filterwarnings("ignore")

In [2]:
# Get today's date
today = date.today()
print(today)
yesterday = today - timedelta(days=1)
print(yesterday)

2023-05-26
2023-05-25


In [3]:
ytddt = yesterday.strftime('%Y%m%d')
ytddt

'20230525'

In [4]:
filepath = "J:/CDMSG/Outgoing/DMT/"
tracker_file_path = 'J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/'
ploan_file_path = 'X:/010 DM Datamarts/101 RAC/004 PLOAN datamarts/'

### Disbursement

In [5]:
# Construct the filename using yesterday's date
df=pd.read_excel(filepath + 'rel_disbursement_'+ str(ytddt)+'.xlsx', dtype={'type1':str})
df

Txn_Date      type1   cnt  disburse_amt         dat    type2  \
0 2023-05-25  2. Reno-i  18.0         438.2         NaN      NaN   
1        NaT        NaN   NaN           NaN  20230525.0  1. Reno   
2        NaT        NaN   NaN           NaN  20230525.0   2. Edu   

   PRINCIPAL_OS_AMT  
0               NaN  
1        144.974681  
2          3.086741

In [6]:
count_edu = 0  # Set the default count value to 0

# Check if there is a row with "type1 1. Edu Loan"
if df['type1'].str.contains('1. Edu').any():
    # Get the count value for "type1 1. Edu Loan" if it exists
    count_edu = df.loc[df['type1'] == '1. Edu', 'cnt'].values[0]

print(count_edu)

0


In [7]:
disburse_edu = 0  # Set the default disburse value to 0

# Check if there is a row with "type1 1. Edu Loan"
if df['type1'].str.contains('1. Edu').any():
    # Get the disburse value for "type1 1. Edu Loan" if it exists
    disburse_edu = df.loc[df['type1'] == '2. Edu', 'disburse_amt'].values[0]

print(disburse_edu)

0


In [8]:
count_reno = 0  # Set the default count value to 0

# Check if there is a row with "type1 1. Edu Loan"
if df['type1'].str.contains('2. Reno-i').any():
    # Get the count value for "type1 1. Edu Loan" if it exists
    count_reno = df.loc[df['type1'] == '2. Reno-i', 'cnt'].values[0]

print(count_reno)

18.0


In [9]:
disburse_reno = 0  # Set the default disburse value to 0

# Check if there is a row with "type1 1. Edu Loan"
if df['type1'].str.contains('2. Reno-i').any():
    # Get the disburse value for "type1 1. Edu Loan" if it exists
    disburse_reno = df.loc[df['type1'] == '2. Reno-i', 'disburse_amt'].values[0]

print(disburse_reno)

438.2


In [10]:
#sum of counts
count_sum = count_edu + count_reno
count_sum

18.0

In [11]:
#sum of disbursement
disburse_sum = disburse_edu + disburse_reno
disburse_sum

438.2

In [12]:
principal_amt_reno_index  = df['PRINCIPAL_OS_AMT'].first_valid_index()

In [13]:
principal_os_amt_reno = df['PRINCIPAL_OS_AMT'][principal_amt_reno_index]
principal_os_amt_reno

144.97468099999998

In [14]:
principal_amt_edu_index  = df['PRINCIPAL_OS_AMT'].last_valid_index()
principal_amt_edu_index

2

In [15]:
principal_os_amt_edu = df['PRINCIPAL_OS_AMT'][principal_amt_edu_index]
principal_os_amt_edu

3.0867408800000007

In [16]:
enr_sum = principal_os_amt_edu + principal_os_amt_reno
enr_sum

148.06142187999998

In [17]:
disbursement_cnt_df = pd.DataFrame([count_sum,0,count_reno,count_edu])
disbursement_cnt_df

0
0  18.0
1   0.0
2  18.0
3   0.0

In [18]:
disbursement_value_df = pd.DataFrame([disburse_sum,0,disburse_reno,disburse_edu])
disbursement_value_df

0
0  438.2
1    0.0
2  438.2
3    0.0

In [19]:
total_renoedu_enr = pd.DataFrame([enr_sum, principal_os_amt_reno, principal_os_amt_edu])
total_renoedu_enr

0
0  148.061422
1  144.974681
2    3.086741

### PLoan

In [20]:
ploan =pd.read_excel("D:/PLoan/001 SLPACQUIDMART.xlsx", index_col=0)
df1 = ploan.copy()
df1

Final Approved Loan Amount  Original Loan Request Amount  \
0                           2000                        2000.0   
1                           8000                        8000.0   
2                          10000                       10000.0   
3                          10000                       10000.0   
4                           4000                        4000.0   
...                          ...                           ...   
3935                       12000                       16000.0   
3936                        4000                        4000.0   
3937                        6000                        6000.0   
3938                        6000                        6000.0   
3939                       10000                           NaN   

     Loan Submission Date Loan Application Status Loan Application Decision  \
0              01/01/2023                end(end)        rejected(rejected)   
1              01/01/2023                end(end)        rejected(rejected)   
2              01/01/2023                end(end)        rejected(rejected)   
3              01/02/2023                end(end)        rejected(rejected)   
4              01/02/2023                end(end)        approved(approved)   
...                   ...                     ...                       ...   
3935           31/03/2023                end(end)        approved(approved)   
3936           31/12/2022                end(end)        approved(approved)   
3937           31/12/2022                end(end)        rejected(rejected)   
3938           31/12/2022                end(end)        rejected(rejected)   
3939                  NaN                     NaN                       NaN   

      Loan Application Status.1         Internal ApplyNo Loan Decision Date  \
0                           NaN  CIMB01012311775Cashloan         01/01/2023   
1                           NaN  CIMB01012314331Cashloan         01/01/2023   
2                           NaN  CIMB01012357694Cashloan         01/01/2023   
3                           NaN  CIMB01022301599Cashloan         04/02/2023   
4                           NaN  CIMB01022304163Cashloan         02/02/2023   
...                         ...                      ...                ...   
3935                        NaN  CIMB31032396421Cashloan         05/04/2023   
3936                        NaN  CIMB31122228882Cashloan         04/01/2023   
3937                        NaN  CIMB31122281223Cashloan         03/01/2023   
3938                        NaN  CIMB31122292996Cashloan         31/12/2022   
3939                        NaN                      NaN                NaN   

      Loan application Labels  Application Revision Count  ...  \
0                         NaN                         0.0  ...   
1                         NaN                         0.0  ...   
2                         NaN                         0.0  ...   
3                         NaN                         0.0  ...   
4                         NaN                         0.0  ...   
...                       ...                         ...  ...   
3935                      NaN                         0.0  ...   
3936                      NaN                         0.0  ...   
3937                      NaN                         0.0  ...   
3938                      NaN                         0.0  ...   
3939                      NaN                         NaN  ...   

      PreviousYrTotalNoa  Monthly Instalment Amount  Loan Account Number(sub)  \
0                57857.0                        182                       NaN   
1                35026.0                        197                       NaN   
2                35026.0                        194                       NaN   
3                31867.0                        235                       NaN   
4                27190.0                         97                       NaN   
...                  ...            

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3940 entries, 0 to 3939
Data columns (total 86 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Final Approved Loan Amount           3940 non-null   int64  
 1   Original Loan Request Amount         3938 non-null   float64
 2   Loan Submission Date                 3939 non-null   object 
 3   Loan Application Status              3939 non-null   object 
 4   Loan Application Decision            3829 non-null   object 
 5   Loan Application Status.1            0 non-null      float64
 6   Internal ApplyNo                     3939 non-null   object 
 7   Loan Decision Date                   3829 non-null   object 
 8   Loan application Labels              0 non-null      float64
 9   Application Revision Count           3939 non-null   float64
 10  orderSubmissionPeriod                0 non-null      float64
 11  Application Resubmission Count

In [22]:
df1['Loan Application Decision'].value_counts()

rejected(rejected)        3021
approved(approved)         603
terminated(terminated)     205
Name: Loan Application Decision, dtype: int64

In [23]:
#converting Loan Submission Date to datetime
df1['Loan Submission Date'] = pd.to_datetime(df1['Loan Submission Date'], format='%d/%m/%Y')
df1['Loan Submission Date']

0      2023-01-01
1      2023-01-01
2      2023-01-01
3      2023-02-01
4      2023-02-01
          ...    
3935   2023-03-31
3936   2022-12-31
3937   2022-12-31
3938   2022-12-31
3939          NaT
Name: Loan Submission Date, Length: 3940, dtype: datetime64[ns]

In [24]:
ytd_df1 = df1.loc[df1['Loan Submission Date'].dt.date == yesterday]
ytd_df1

Final Approved Loan Amount  Original Loan Request Amount  \
3248                       90000                       90000.0   
3249                       20000                       20000.0   
3250                        5000                        5000.0   
3251                       10000                       10000.0   
3252                       10000                       10000.0   
...                          ...                           ...   
3311                       15000                       15000.0   
3312                        3000                        3000.0   
3313                       32800                       60000.0   
3314                       10000                       10000.0   
3315                       10000                       10000.0   

     Loan Submission Date                 Loan Application Status  \
3248           2023-05-25                                end(end)   
3249           2023-05-25  approvalInProgress(approvalInProgress)   
3250           2023-05-25                                end(end)   
3251           2023-05-25  approvalInProgress(approvalInProgress)   
3252           2023-05-25  approvalInProgress(approvalInProgress)   
...                   ...                                     ...   
3311           2023-05-25  approvalInProgress(approvalInProgress)   
3312           2023-05-25                                end(end)   
3313           2023-05-25                                end(end)   
3314           2023-05-25                                end(end)   
3315           2023-05-25                                end(end)   

     Loan Application Decision  Loan Application Status.1  \
3248        rejected(rejected)                        NaN   
3249                       NaN                        NaN   
3250        rejected(rejected)                        NaN   
3251                       NaN                        NaN   
3252                       NaN                        NaN   
...                        ...                        ...   
3311                       NaN                        NaN   
3312        rejected(rejected)                        NaN   
3313        approved(approved)                        NaN   
3314        rejected(rejected)                        NaN   
3315        rejected(rejected)                        NaN   

             Internal ApplyNo Loan Decision Date  Loan application Labels  \
3248  CIMB25052300280Cashloan         25/05/2023                      NaN   
3249  CIMB25052300282Cashloan                NaN                      NaN   
3250  CIMB25052302359Cashloan         25/05/2023                      NaN   
3251  CIMB25052303001Cashloan                NaN                      NaN   
3252  CIMB25052303970Cashloan                NaN                      NaN   
...                       ...                ...                      ...   
3311  CIMB25052395847Cashloan                NaN                      NaN   
3312  CIMB25052396204Cashloan         25/05/2023                      NaN   
3313  CIMB25052396936Cashloan         25/05/2023                      NaN   
3314  CIMB25052397883Cashloan         25/05/2023                      NaN   
3315  CIMB25052399416Cashloan         25/05/2023                      NaN   

      Application Revision Count  ...  PreviousYrTotalNoa  \
3248                         0.0  ...            287672.0   
3249                         0.0  ...             62860.0   
3250                         0.0  ...                 0.0   
3251                         0.0  ...             52656.0   
3252                         0.0  ...             29005.0   
...                          ...  ...                 ...   
3311                         0.0  ...             34940.0   
3312                         0.0  ...                 0.0   
3313                         0.0  ...             84062.0   
3314                         0.0  ...            129120.0   
3315                         0.0  ...             35000

In [25]:
ytd_df1['Loan Application Decision'].unique()

array(['rejected(rejected)', nan, 'approved(approved)'], dtype=object)

In [26]:
ytd_df1['Loan Application Decision'].value_counts()

rejected(rejected)    49
approved(approved)     1
Name: Loan Application Decision, dtype: int64

In [27]:
filtered_df = ytd_df1[ytd_df1['Loan Application Status'] == 'end(end)']

In [28]:
valid_values = ['rejected(rejected)', 'approved(approved)', 'terminated(terminated)']

In [29]:
filtered_df1 = filtered_df[filtered_df['Loan Application Decision'].isin(valid_values)]
processed_applications = filtered_df1.groupby('Loan Application Decision').size().sum()
processed_applications

50

In [30]:
total_applications = len(ytd_df1)
total_applications

68

In [31]:
#converting Loan Decision Date to datetime
df1['Loan Decision Date'] = pd.to_datetime(df1['Loan Decision Date'], format='%d/%m/%Y')
df1['Loan Decision Date']

0      2023-01-01
1      2023-01-01
2      2023-01-01
3      2023-02-04
4      2023-02-02
          ...    
3935   2023-04-05
3936   2023-01-04
3937   2023-01-03
3938   2022-12-31
3939          NaT
Name: Loan Decision Date, Length: 3940, dtype: datetime64[ns]

In [32]:
ytd_df2 = df1.loc[df1['Loan Decision Date'].dt.date == yesterday]
ytd_df2

Final Approved Loan Amount  Original Loan Request Amount  \
2593                       11500                       12000.0   
2614                       35000                       35000.0   
2997                        3600                        4000.0   
3117                       12000                       12000.0   
3132                        4000                        4000.0   
3140                       16000                       16000.0   
3149                       15000                       15000.0   
3157                       10000                       10000.0   
3248                       90000                       90000.0   
3250                        5000                        5000.0   
3253                       51000                       51000.0   
3254                        8000                        8000.0   
3255                        4000                        4000.0   
3256                        8000                        8000.0   
3257                       10000                       10000.0   
3258                        5000                        5000.0   
3260                        8000                        8000.0   
3261                       10000                       10000.0   
3263                       16000                       16000.0   
3266                       90000                       90000.0   
3267                      100000                      100000.0   
3268                       10000                       10000.0   
3269                        5000                        5000.0   
3270                        8000                        8000.0   
3271                       20000                       20000.0   
3273                       21000                       21000.0   
3275                        8000                        8000.0   
3277                       10000                       10000.0   
3278                       60000                       60000.0   
3280                       15000                       15000.0   
3281                       18000                       18000.0   
3284                       22000                       22000.0   
3285                        6000                        6000.0   
3286                       10000                       10000.0   
3289                        3000                        3000.0   
3291                       10000                       10000.0   
3294                       16000                       16000.0   
3297                       35000                       35000.0   
3301                        5000                        5000.0   
3302                       20000                       20000.0   
3303                        5000                        5000.0   
3304                      100000                      100000.0   
3306                       10000                       10000.0   
3307                        4000                        4000.0   
3309                        3000                        3000.0   
3310                        5000                        5000.0   
3312                        3000                        3000.0   
3313                       32800                       60000.0   
3314                       10000                       10000.0   
3315                       10000                       10000.0   

     Loan Submission Date Loan Application Status Loan Application Decision  \
2593           2023-05-20                end(end)        approved(approved)   
2614           2023-05-20                end(end)    terminated(terminated)   
2997           2023-05-23                end(end)        approved(approved)   
3117           2023-05-24                end(end)        rejected(rejected)   
3132           2023-05-24                end(end)        rejected(rejected)   
3140           2023-05-24                end(end)        rejected(rejected)   
3149           2023-05-24                end(end)        rejected(rejected)   
3157   

In [33]:
filtered_df_LD = ytd_df2[ytd_df2['Loan Application Status'] == 'end(end)']

In [34]:
approved_applications = len(filtered_df_LD.loc[filtered_df_LD['Loan Application Decision'] == 'approved(approved)'])
approved_applications

3

In [35]:
approved_sum = filtered_df_LD.loc[filtered_df_LD['Loan Application Decision'] == 'approved(approved)']['Final Approved Loan Amount'].sum()
approved_sum

47900

In [36]:
requested_sum = filtered_df_LD.loc[filtered_df_LD['Loan Application Decision'] == 'approved(approved)']['Original Loan Request Amount'].sum()
requested_sum

76000.0

In [37]:
approval_rate = approved_applications/processed_applications
approval_rate

if pd.isna(approval_rate):
    approval_rate = 0
print(approval_rate)

0.06


In [38]:
applied_v_approved_amt = approved_sum/requested_sum
applied_v_approved_amt

if pd.isna(applied_v_approved_amt):
    applied_v_approved_amt = 0
print(applied_v_approved_amt)

0.6302631578947369


In [39]:
consolidated_df = pd.DataFrame([total_applications, processed_applications, approved_applications, requested_sum, approved_sum, approval_rate, applied_v_approved_amt])
consolidated_df

0
0     68.000000
1     50.000000
2      3.000000
3  76000.000000
4  47900.000000
5      0.060000
6      0.630263

### Copying

In [40]:
dailytracker = openpyxl.load_workbook('J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/Daily Tracker_202305.xlsx')

In [41]:
dailytracker_sheet = dailytracker['SUMMARY - Card level Jan 2023']

In [42]:
column_letter = openpyxl.utils.get_column_letter(yesterday.day + 2)
column_letter

'AA'

In [43]:
dailytracker_sheet[column_letter + '115'].value = disbursement_cnt_df.iloc[0,0]
dailytracker_sheet[column_letter + '116'].value = disbursement_cnt_df.iloc[1,0]
dailytracker_sheet[column_letter + '117'].value = disbursement_cnt_df.iloc[2,0]
dailytracker_sheet[column_letter + '118'].value = disbursement_cnt_df.iloc[3,0]

In [44]:
dailytracker_sheet[column_letter + '121'].value = disbursement_value_df.iloc[0,0]
dailytracker_sheet[column_letter + '122'].value = disbursement_value_df.iloc[1,0]
dailytracker_sheet[column_letter + '123'].value = disbursement_value_df.iloc[2,0]
dailytracker_sheet[column_letter + '124'].value = disbursement_value_df.iloc[3,0]

In [45]:
dailytracker_sheet[column_letter + '127'].value = total_renoedu_enr.iloc[0,0]
dailytracker_sheet[column_letter + '128'].value = total_renoedu_enr.iloc[1,0]
dailytracker_sheet[column_letter + '129'].value = total_renoedu_enr.iloc[2,0]

In [46]:
dailytracker_sheet[column_letter + '131'].value = consolidated_df.iloc[0,0]
dailytracker_sheet[column_letter + '132'].value = consolidated_df.iloc[1,0]
dailytracker_sheet[column_letter + '133'].value = consolidated_df.iloc[2,0]
dailytracker_sheet[column_letter + '137'].value = consolidated_df.iloc[3,0]
dailytracker_sheet[column_letter + '138'].value = consolidated_df.iloc[4,0]
dailytracker_sheet[column_letter + '134'].value = consolidated_df.iloc[5,0]
dailytracker_sheet[column_letter + '139'].value = consolidated_df.iloc[6,0]

In [47]:
dailyworking = openpyxl.load_workbook('J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/DailyWorking v2.xlsx', data_only=True)

In [48]:
dailyworking_sheet = dailyworking['To Python']

In [49]:
# Copy the range A1:A102 from source to destination file
for row in range(2, 103):
    cell_value = dailyworking_sheet.cell(row=row, column=1).value
    dailytracker_sheet.cell(row=row+10, column=openpyxl.utils.column_index_from_string(column_letter)).value = cell_value
    dailytracker_sheet.value = cell_value

In [50]:
#Every start of the month please double check the column letter is the one that contains 'MTD MM'  
dailytracker_sheet['AH13'].value = dailyworking_sheet.cell(3,2).value

In [51]:
#backtrack start here again
dailytracker.save('J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/Daily Tracker_202305.xlsx')

In [52]:
#rewrite code using xlwings
dailytracker_copy = xw.Book('J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/Daily Tracker_202305.xlsx', update_links=False)

In [53]:
ccpfdaily = xw.Book('J:/Decision Management/Regular Report/1-Daily/Cards_DailyTracker/CCPF Daily Tracker_202305.xlsx', update_links=False)

In [54]:
dailytracker_copy_sheet = dailytracker_copy.sheets['SUMMARY - Card level Jan 2023']
ccpfdaily_sheet = ccpfdaily.sheets['SUMMARY - Card level Dec 22']

In [55]:
start = today.replace(day=1)

column_letter2 = xw.utils.col_name(start.day + 2)

###################################################################

# Set the limit to column 'BB'
limit_column = 'BB'

# Get the range from row 9 to the limit column
range_str = f'AJ9:BB9'

# Get the last non-empty cell in row 9 within the limit
last_cell = dailytracker_copy_sheet.range(range_str).end('right')
column_number = last_cell.column
column_letter3 = xw.utils.col_name(column_number)


# Print the result
print(column_letter2, column_letter3)

C AY


In [56]:
start_row = 10
end_row = 139

# Get the source daily tracker range to copy
source_daily_tracker_range = f'{column_letter2}{start_row}:{column_letter3}{end_row}'
print(source_daily_tracker_range)
    
# Get the destination ccpf range to paste
destination_ccpf_range = f'{column_letter2}{start_row}:{column_letter3}{end_row}'
print(destination_ccpf_range)

C10:AY139
C10:AY139


In [57]:
#copy
ccpfdaily_sheet.range(destination_ccpf_range).value = dailytracker_copy_sheet.range(source_daily_tracker_range).value

In [58]:
#copy dates
dailytracker_date = dailytracker_copy.sheets[0]['B1:B6']
ccpf_date = ccpfdaily.sheets[0]['B1:B6']


dailytracker_date.copy()
ccpf_date.paste(paste = 'values')

In [59]:
dailytracker_copy.save()
dailytracker_copy.close()
ccpfdaily.save()
ccpfdaily.close()